In [28]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Ha\'.wav"
template_folder = "../Dataset/01.Ha\'/"
output_file = '../hasil_testing/rafi_ha.txt'

template_score = 19.808
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M1_11.wav: 16.27128677063952

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M1_12.wav: 14.444416514840643

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M1_13.wav: 17.079251197834957

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M2_11.wav: 18.790888746640096

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M2_12.wav: 15.865714538426657

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M2_13.wav: 16.53357041543621

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M3_1.wav: 24.637520538055544

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M3_2.wav: 24.639386154319872

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha'.wav dan ../Dataset/01.Ha'/Ha'_M3_3.wav: 25.543656848838435

Jarak 

In [30]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Kha\'.wav"
template_folder = "../Dataset/02.Kha\'/"
output_file = '../hasil_testing/rafi_kha.txt'

template_score = 20.167
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M1_11.wav: 19.348973516922978

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M1_12.wav: 18.745518512123166

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M1_13.wav: 20.22127261462022

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M2_11.wav: 23.828842237212744

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M2_12.wav: 22.687999177499687

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M2_13.wav: 22.476052891898846

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M3_1.wav: 21.654591109036822

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M3_2.wav: 21.460421692675585

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Kha'.wav dan ../Dataset/02.Kha'/Kha'_M3_3.wav

In [31]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Shad.wav"
template_folder = "../Dataset/03.Shad/"
output_file = '../hasil_testing/rafi_shad.txt'

template_score = 23.004
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M1_11.wav: 20.73135199811801

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M1_12.wav: 24.734574729101492

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M1_13.wav: 19.67883274667005

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M2_11.wav: 17.953404668124325

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M2_12.wav: 17.54334339307232

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M2_13.wav: 22.153279895315976

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M3_1.wav: 40.06918573272033

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M3_2.wav: 30.151895653503693

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Shad.wav dan ../Dataset/03.Shad/Shad_M3_3.wav: 2

In [34]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Dhad.wav"
template_folder = "../Dataset/04.Dhad/"
output_file = '../hasil_testing/rafi_dhad.txt'

template_score = 19.820
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M1_11.wav: 16.76947854953783

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M1_12.wav: 15.4620739288776

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M1_13.wav: 17.88529065114493

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M2_11.wav: 18.451882609822885

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M2_12.wav: 16.61771315680756

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M2_13.wav: 18.00814703968026

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M3_1.wav: 21.102536031246917

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M3_2.wav: 23.244183004427303

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhad.wav dan ../Dataset/04.Dhad/Dhad_M3_3.wav: 23.

In [35]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Tha\'.wav"
template_folder = "../Dataset/05.Tha\'/"
output_file = '../hasil_testing/rafi_tha.txt'

template_score = 19.670
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M1_11.wav: 23.256014249513125

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M1_12.wav: 20.09545924579308

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M1_13.wav: 19.082654299068047

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M2_11.wav: 23.232418784701267

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M2_12.wav: 24.41931868433785

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M2_13.wav: 20.285841120410666

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M3_1.wav: 19.39533244838986

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M3_2.wav: 29.892384528591347

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Tha'.wav dan ../Dataset/05.Tha'/Tha'_M3_3.wav: 

In [36]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Dhza\'.wav"
template_folder = "../Dataset/06.Dhza\'/"
output_file = '../hasil_testing/rafi_dhza.txt'

template_score = 22.409
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Dataset/06.Dhza'/Dhza'_M1_11.wav: 23.642053412710563

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Dataset/06.Dhza'/Dhza'_M1_12.wav: 22.422992500095148

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Dataset/06.Dhza'/Dhza'_M1_13.wav: 21.324953064172675

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Dataset/06.Dhza'/Dhza'_M2_11.wav: 29.640612626845744

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Dataset/06.Dhza'/Dhza'_M2_12.wav: 25.31022126924674

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Dataset/06.Dhza'/Dhza'_M2_13.wav: 24.547042143081608

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Dataset/06.Dhza'/Dhza'_M3_1.wav: 34.42699100635547

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Dataset/06.Dhza'/Dhza'_M3_2.wav: 28.250299030561614

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Dhza'.wav dan ../Datas

In [37]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_\'Ain.wav"
template_folder = "../Dataset/07.\'Ain/"
output_file = '../hasil_testing/rafi_ain.txt'

template_score = 26.060
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M1_11.wav: 21.85861337213965

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M1_12.wav: 21.315516178983078

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M1_13.wav: 19.663661410102613

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M2_11.wav: 23.41734419860216

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M2_12.wav: 21.366506208965685

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M2_13.wav: 24.252856926848978

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M3_1.wav: 26.66479971531539

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M3_2.wav: 26.95910269803723

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_'Ain.wav dan ../Dataset/07.'Ain/'Ain_M3_3.wav: 2

In [38]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Ghain.wav"
template_folder = "../Dataset/08.Ghain/"
output_file = '../hasil_testing/rafi_ghain.txt'

template_score = 21.120
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Dataset/08.Ghain/Ghain_M1_11.wav: 18.832770732900627

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Dataset/08.Ghain/Ghain_M1_12.wav: 17.802134234711442

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Dataset/08.Ghain/Ghain_M1_13.wav: 21.262647521623812

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Dataset/08.Ghain/Ghain_M2_11.wav: 24.049418195702266

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Dataset/08.Ghain/Ghain_M2_12.wav: 22.184655652932577

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Dataset/08.Ghain/Ghain_M2_13.wav: 21.86316009809999

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Dataset/08.Ghain/Ghain_M3_1.wav: 23.010724090572868

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Dataset/08.Ghain/Ghain_M3_2.wav: 19.215459557845612

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ghain.wav dan ../Data

In [39]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Qaf.wav"
template_folder = "../Dataset/09.Qaf/"
output_file = '../hasil_testing/rafi_qaf.txt'

template_score = 21.357
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M1_11.wav: 29.972470715158657

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M1_12.wav: 19.528963004245245

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M1_13.wav: 17.864137378216277

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M2_11.wav: 20.026429312423033

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M2_12.wav: 21.597899595886897

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M2_13.wav: 18.14006722431769

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M3_1.wav: 20.29775632140197

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M3_2.wav: 24.661484629865186

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Qaf.wav dan ../Dataset/09.Qaf/Qaf_M3_3.wav: 25.428007163136737

Jarak 

In [40]:
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = nr.reduce_noise(y, sr)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T


def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    counts = 0
    total_distances = 0
    final_score = 0

    with open(output_file, 'w') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = extraction(audio_template)

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances = total_distances + dtw_distance
            counts = counts + 1

            result_line = f"Jarak DTW antara {audio_input} dan {audio_template}: {dtw_distance}\n"
            print(result_line)
            f.write(result_line)

        # hitung total distance
        write_total_distance = f"Total Distance = {total_distances}\n"        
        print(write_total_distance)
        f.write(write_total_distance)

        # hitung rata rata dari total distance
        average_distance = total_distances / counts
        write_average_distance = f"Average Distance = {average_distance}\n"
        print(write_average_distance)
        f.write(write_average_distance)

        # hitung selisih antara rata rata distance dengan template
        dif_distance = average_distance - template_score
        write_dif_distance = f'Difference Distance = {dif_distance}\n'
        print(write_dif_distance)
        f.write(write_dif_distance)

        # perhitungan score
        if (dif_distance <= 0):
            final_score = 100
        elif(dif_distance >= max_threshold_score ):
            final_score = 0
        else:
            final_score = int(100 - ((dif_distance/max_threshold_score) * 100))
        
        write_final_score = f'Final Score = {final_score}\n'
        print(write_final_score)
        f.write(write_final_score)

            


input_file_audio= "../Dataset/Testing_Rafi/rafi_Ha^\'.wav"
template_folder = "../Dataset/10.Ha^\'/"
output_file = '../hasil_testing/rafi_he.txt'

template_score = 20.693
max_threshold_score = 5

main(input_file_audio, template_folder, output_file, template_score, max_threshold_score)

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M1_11.wav: 19.875850356052187

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M1_12.wav: 19.947874663794124

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M1_13.wav: 22.574754689133236

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M2_11.wav: 22.51082035490891

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M2_12.wav: 26.183377380351928

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M2_13.wav: 24.179977361834887

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M3_1.wav: 20.35911689343391

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M3_2.wav: 19.339239168320407

Jarak DTW antara ../Dataset/Testing_Rafi/rafi_Ha^'.wav dan ../Dataset/10.Ha^'/Ha^'_M3_3.wav: